## 1. Online Retail Data Set
### 1.1 Dataset Download
The Online Retail Data Set is a dataset made available on the UCI Machine Learning repository. Itwhich contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based online retail.

It is available as a CSV file at the following URL.
https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv

Each of the 541,909 rows contains an item that has been purchased by someone. Items can be grouped into invoices (you can think of these as receipts), where each invoice has been issued for a specific buyer, and can contain multiple items.
The columns contained in the CSV file are the following:
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter “C”, it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each
distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

As an example, the following lines all refer to the same invoice (InvoiceNo = 574021).

574021,23301,GARDENERS KNEELING PAD KEEP CALM ,48,2011-11-02 12:18:00, 1.45, 14434, United Kingdom

574021,23355,HOT WATER BOTTLE KEEP CALM,24,2011-11-02 12:18:00, 4.15, 14434, United Kingdom

574021,23284,DOORMAT KEEP CALM AND COME IN,20,2011-11-02 12:18:00, 7.08, 14434, United Kingdom

### 1.2. Exercises
**1.** 1. First, you need to load the dataset into memory, using the csv module. Make sure you identify all valid rows. Also consider that rows having an InvoiceNo that starts with C should be discarded, as they indicate that the invoice is about a cancelled purchase.

In [1]:
import csv

In [2]:
with open('online_retail.csv', mode="r") as file:
    iterator = csv.reader(file)
    columns = next(iterator)
    dataset = [[] for i in columns]
    for i in iterator:
        if i[0][0].upper() != 'C':
            for j in range(len(i)):
                dataset[j].append(i[j])

In [3]:
#in order to be sure that all columns are correctly appended
for i in range(len(columns)):
    print(len(dataset[i]),",", dataset[i][:2])

532621 , ['536365', '536365']
532621 , ['85123A', '71053']
532621 , ['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN']
532621 , ['6', '6']
532621 , ['12/1/2010 8:26', '12/1/2010 8:26']
532621 , ['2.55', '3.39']
532621 , ['17850', '17850']
532621 , ['United Kingdom', 'United Kingdom']


**2.** Now that you have a dataset of items, you should aggregate it at an “invoice” level. For each invoice (identified by InvoiceNo) there can be multiple items (from multiple rows) in the dataset. For each invoice, you should build a list of all items belonging to it. For the example invoice presented in 1.1, you want to build the following list:

[ "GARDENERS KNEELING PAD KEEP CALM",

"HOT WATER BOTTLE KEEP CALM",

"DOORMAT KEEP CALM AND COME IN" ]


In [4]:
aggdata = [[] for i in columns]
transactionNum = len(dataset[0])
columnNum = len(aggdata)
for i in range(transactionNum):
    if dataset[0][i] not in aggdata[0]:
        for c in range(columnNum):
            if c != 2:
                aggdata[c].append(dataset[c][i])
            else:
                aggdata[c].append([dataset[c][i]])
    else:
        index = aggdata[0].index(dataset[0][i])
        aggdata[2][index].append(dataset[2][i])

In [5]:
#in order to be sure that data is correctly aggregated
for i in range(len(columns)):
    print(len(aggdata[i]),",", aggdata[i][:2])

22064 , ['536365', '536366']
22064 , ['85123A', '22633']
22064 , [['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER'], ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT']]
22064 , ['6', '6']
22064 , ['12/1/2010 8:26', '12/1/2010 8:28']
22064 , ['2.55', '1.85']
22064 , ['17850', '17850']
22064 , ['United Kingdom', 'United Kingdom']


In [6]:
itemsUni = sorted(list({i for i in dataset[2]}))
len(itemsUni)

4208

**3.** You should now have a list (one for each invoice) of lists (each list containing the items bought for that invoice). Now, we need to convert this into a matrix form. Of the many possible formats, we will use the one expected by the Mlxtend library, which is as follows. Given an ordered list of M possible items (in this case, all possible products that can be bought), and given N itemsets (in this case, invoices), we should build a matrix of N rows and M columns. The element at the i th row and jth column should be 1 if the ith itemset (invoice) contains the jth item (product), 0 otherwise

Once we have defined this matrix (as a list of lists), we can use Pandas to convert it to a DataFrame.

In [7]:
import numpy as np

In [8]:
matrix = np.zeros(shape=(len(aggdata[0]),len(itemsUni)),dtype = int)

In [9]:
for i in range(len(aggdata[2])):
    for item in aggdata[2][i]:
        index = itemsUni.index(item)
        matrix[i][index] = 1

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(matrix,index=aggdata[0],columns=itemsUni)
df.loc["574021", df.loc["574021"]==1]

DOORMAT KEEP CALM AND COME IN        1
GARDENERS KNEELING PAD KEEP CALM     1
HOT WATER BOTTLE KEEP CALM           1
Name: 574021, dtype: int32

**4.** With the df that you defined in the previous exercise, you can now use the fp_growth function. The first argument required is the previously built DataFrame, df. The second is the minimum support (minsup), i.e. the minimum fraction of the entire dataset in which the itemset should show up for it to be considered “frequent”. Try using different values of minsup, such as 0.5, 0.1, 0.05, 0.02, 0.01. How many results do you obtain as minsup varies? You can check the number of frequent itemsets identified and print them all with the following code snipped:

fi = fpgrowth(df, 0.05)

print(len(fi))

print(fi.to_string())


In [12]:
import sys
!{sys.executable} -m pip install mlxtend

In [13]:
import mlxtend 
from mlxtend.frequent_patterns import fpgrowth

In [14]:
import warnings
warnings.filterwarnings('ignore')
minsups = [0.5, 0.1, 0.05, 0.02, 0.01]
for ms in minsups:
    print("---------------------------------")
    fi = fpgrowth(df, ms)
    print(len(fi))
    print(fi.to_string())

---------------------------------
0
Empty DataFrame
Columns: [support, itemsets]
Index: []
---------------------------------
1
    support itemsets
0  0.102429   (3904)
---------------------------------
23
     support itemsets
0   0.102429   (3904)
1   0.065945    (244)
2   0.051351   (1825)
3   0.051033   (2743)
4   0.052574   (2431)
5   0.070885   (2046)
6   0.059826   (2387)
7   0.055203   (1856)
8   0.053662   (1871)
9   0.052665   (1824)
10  0.053254   (1869)
11  0.056608   (2245)
12  0.054433   (1702)
13  0.094815   (1858)
14  0.052438   (2049)
15  0.057696   (2038)
16  0.052121   (2041)
17  0.065899      (0)
18  0.090147   (2905)
19  0.050625   (2802)
20  0.076414   (2463)
21  0.062772   (3185)
22  0.051713   (3514)
---------------------------------
303
      support            itemsets
0    0.102429              (3904)
1    0.020803              (1902)
2    0.021710              (1645)
3    0.065945               (244)
4    0.035216              (1738)
5    0.028508           

In [15]:
#name of the item
print(itemsUni[3904]) 
#control to see it whether the item exist 10.24% of transactions
print('%',100*df.iloc[:,3904].sum()/len(df),sep='') 

WHITE HANGING HEART T-LIGHT HOLDER
%10.242929659173313


**5.** Consider the itemsets extracted for minsup = 0.02. How many items are contained? Which ones would you consider to be the most useful?

In [16]:
fi = fpgrowth(df, 0.02)
print(len(fi))
print(fi[:5].to_string())

303
    support itemsets
0  0.102429   (3904)
1  0.020803   (1902)
2  0.021710   (1645)
3  0.065945    (244)
4  0.035216   (1738)


In [17]:
#we know that there are 303 rows in the dataframe. some of them contain 1 item while
#others might contain more. let's examine if there is any itemset which contain more
frequent_itemset = fi[fi['itemsets'].map(len)>1]
print(f"There are {len(frequent_itemset)} itemsets which contain more than 1 item")
frequent_itemset
#Since the invocices are mostly consist of more than 1 item, this kind of information
#might be more useful for business practices. For example, you can recommend other products
#while a customer buy the other one.

There are 57 itemsets which contain more than 1 item


,support,itemsets
246,0.021392,"(1824, 1825)"
247,0.029007,"(162, 166)"
248,0.021302,"(165, 166)"
249,0.024429,"(3970, 3966)"
250,0.022435,"(3904, 2837)"
251,0.026242,"(1858, 2046)"
252,0.037391,"(1856, 1858)"
253,0.023341,"(1856, 1871)"
254,0.032814,"(1858, 1871)"
255,0.026514,"(1858, 1846)"


**6.** Select one of the frequent itemsets returned by *fpgrowth*. Use this itemset to extract the relevant association rules. For these rules, compute their confidence.

In [18]:
#select one of them randomly: (2656,1599)
#We can extract the two rules: 2656 => 1599 and 1599 => 2656 (confidence)

In [19]:
support2656 = df.iloc[:,2656].sum()/len(df)
support1599 = df.iloc[:,1599].sum()/len(df)
supportboth = len(df.loc[(df[list(df.columns)[2656]]==1) & 
                     (df[list(df.columns)[1599]]==1),
                     [list(df.columns)[2656],list(df.columns)[1599]]]) / len(df)
print(f"Confidence of 2656 => 1599 is {supportboth/support2656}")
print(f"Confidence of 1599 => 2656 is {supportboth/support1599}")

Confidence of 2656 => 1599 is 0.8263707571801567
Confidence of 1599 => 2656 is 0.6236453201970443


**7.** Extract the association rules from the frequent itemsets extracted with minsup = 0.01. You can find the documentation for association_rules() on the official documentation. You can use the confidence as the metric to identify the rules, and a minimum threshold of 0.85 (feel free to vary these values and observe how the results vary).

In [21]:
from mlxtend.frequent_patterns import association_rules

In [22]:
ms01 = fpgrowth(df, min_support=0.01)

In [28]:
association_rules(ms01,metric='confidence',min_threshold=.85)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(3547, 723)",(2861),0.017177,0.046864,0.014775,0.860158,18.354481,0.013970,6.815824
1,"(3547, 724, 723)",(2861),0.012282,0.046864,0.011104,0.904059,19.291256,0.010528,9.934613
2,"(3547, 3981, 723)",(2861),0.011920,0.046864,0.010968,0.920152,19.634657,0.010409,11.936898
3,"(3547, 724, 3981)",(2861),0.013733,0.046864,0.011784,0.858086,18.310257,0.011140,6.716286
4,"(1856, 1869, 1871)",(1858),0.014005,0.094815,0.012146,0.867314,9.147426,0.010819,6.822003
5,"(3338, 3292)",(3339),0.013416,0.023885,0.012011,0.895270,37.482435,0.011690,9.320323
6,(1731),(1732),0.010877,0.010741,0.010016,0.920833,85.726864,0.009899,12.495897
7,(1732),(1731),0.010741,0.010877,0.010016,0.932489,85.726864,0.009899,14.651378
8,"(723, 724, 3981)",(2861),0.013234,0.046864,0.011376,0.859589,18.342333,0.010756,6.788190
9,"(3561, 1858)",(1092),0.012781,0.032088,0.011285,0.882979,27.517009,0.010875,8.271244
